In [28]:
import os
import jieba
import jieba.analyse
import pandas as pd
import pyodbc
import configparser
from bs4 import BeautifulSoup
import re
import nltk

In [29]:
config = configparser.ConfigParser()
config.read('config.env')
db_UserName = config.get('DEFAULT', 'DB_USERNAME')
db_Password = config.get('DEFAULT', 'DB_PASSWORD')
db_Name = config.get('DEFAULT', 'DB_NAME')
db_Host = config.get('DEFAULT', 'DB_HOST')

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            f"Server={db_Host};"
            f"Database={db_Name};"
            f"UID={db_UserName};"
            f"PWD={db_Password};")

cnxn = pyodbc.connect(cnxn_str)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [30]:
# SQL查詢語句
query = ("select source,id,title ,context from ("
         "select a.source,a.id,title,context from pttpost_referendum_1 a "
         " inner join pttpost b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select a.source,a.id,title,context from pttpost_referendum_1 a "
         " inner join pttpostgossing b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select a.source,convert(varchar,a.id),title,content from dcard.dbo.pttpost_referendum_1 a "
         " inner join dcard.dbo.post b on a.source=b.forum and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.content like '%'+keyname+'%')) "
         " ) m "
         "where 1=1")

# 讀取資料表
df = pd.read_sql(query, cnxn)

c:\Users\HuanChen\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [31]:
# 載入知網詞庫
jieba.set_dictionary('C:\project\python\dict.big5.txt')

# 載入自定義詞庫
jieba.load_userdict('C:\project\python\main.txt')

Building prefix dict from C:\project\python\dict.big5.txt ...
Loading model from cache C:\Users\HuanChen\AppData\Local\Temp\jieba.u7bf78fb8a3e5c528afaa2a9a1de33675.cache
Loading model cost 2.351 seconds.
Prefix dict has been built successfully.


In [32]:
def get_score(article_string):
    with open('NTUSD_positive_unicode.txt', encoding='utf-8', mode='r') as f:
        positive_words = []
        for l in f:
            positive_words.append(l.strip())

    with open('NTUSD_negative_unicode.txt', encoding='utf-8', mode='r') as f:
        negative_words = []
        for l in f:
            negative_words.append(l.strip())

    score = 0

    if (article_string.strip() != ""):
        jieba_result = jieba.cut(article_string, cut_all=False, HMM=True)

        for word in jieba_result:
            if word.strip() != "":
                if word in positive_words:
                    score += 1
                elif word in negative_words:
                    score -= 1
                else:
                    pass

    return score

In [33]:
# 迴圈處理
try:
    for index, row in df.iterrows():
        context = row["context"]
        # 進行相關處理
        score = get_score(context)
        print(row["id"]+f'總分:{score}')
        if (row["source"]!="talk"):
            sql = "update pttpost_referendum_1 set emotionpost=? where source=? and id=?"
        else:
            sql = "update dcard.dbo.pttpost_referendum_1 set emotionpost=? where source=? and id=?"    
        
        cursor.execute(sql, score, row["source"], row["id"])
        cnxn.commit()
finally:
    cnxn.commit()
    cnxn.close()

M.1609433043.A.55E總分:2
M.1609512481.A.550總分:-1
M.1609493102.A.0D7總分:1
M.1609475812.A.E58總分:6
M.1609516575.A.7ED總分:0
M.1609576721.A.650總分:-2
M.1609558726.A.5F8總分:3
M.1609501739.A.70D總分:4
M.1609506912.A.D3E總分:3
M.1609490622.A.B0A總分:-2
M.1609554769.A.F05總分:-2
M.1609487249.A.3CD總分:9
M.1609596105.A.6A9總分:-1
M.1609641857.A.217總分:3
M.1609505423.A.2F4總分:9
M.1609733351.A.BC4總分:0
M.1609749772.A.55E總分:-3
M.1609845593.A.5CD總分:-3
M.1609660066.A.86F總分:-4
M.1609848999.A.08E總分:0
M.1609851332.A.37A總分:-3
M.1609663595.A.43F總分:-4
M.1609660242.A.AE9總分:1
M.1609599419.A.F63總分:-1
M.1609764521.A.A3B總分:-2
M.1609704117.A.6ED總分:-2
M.1609720329.A.B16總分:-6
M.1609730853.A.446總分:3
M.1609638412.A.618總分:-2
M.1609743481.A.B2A總分:-1
M.1609787119.A.E52總分:-9
M.1609697826.A.DFE總分:-2
M.1610079612.A.698總分:-17
M.1609683054.A.8DF總分:2
M.1609831481.A.A89總分:-3
M.1609892019.A.013總分:-7
M.1609675051.A.112總分:2
M.1609752556.A.70B總分:-11
M.1609654111.A.EB0總分:-2
M.1609691141.A.C65總分:6
M.1609713416.A.28F總分:-2
M.1610018732.A.30E總分:-2
M.16101